In [12]:
import pandas as pd

In [13]:
devd = pd.read_csv('Dev Data/subtaskB_dev_data.csv', index_col = 0)
traind = pd.read_csv('Training  Data/subtaskB_data_all.csv', index_col = 0)
triald = pd.read_csv('Trial Data/taskB_trial_data.csv', index_col = 0)

test = pd.read_csv('Test Data/subtaskB_test_data.csv')

deva = pd.read_csv('Dev Data/subtaskB_gold_answers.csv', index_col = 0)
traina = pd.read_csv('Training  Data/subtaskB_answers_all.csv', index_col = 0)
triala = pd.read_csv('Trial Data/taskB_trial_answer.csv', index_col = 0)

In [14]:
train = pd.merge(traind, traina, on='id', how='inner')
trial = pd.merge(triald, triala, on='id', how='inner')
dev = pd.merge(devd, deva, on='id', how = 'inner')

train['id'] = train.index
dev['id'] = dev.index
trial['id'] = trial.index

In [15]:
def func(df):  
    
    col_names=['text', 'klass']
    
    temp = pd.DataFrame()
    temp = pd.DataFrame(columns=col_names)

    for index,row in df.iterrows():
        arow={'klass':row['ans'],'text':str(row['FalseSent']+'<eol>.'+str(row['OptionA']))}
        temp = temp.append(arow, ignore_index=True)
        brow={'klass':row['ans'],'text':str(row['FalseSent']+'<eol>.'+str(row['OptionB']))}
        temp = temp.append(brow, ignore_index=True)
        crow={'klass':row['ans'],'text':str(row['FalseSent']+'<eol>.'+str(row['OptionC']))}
        temp = temp.append(crow, ignore_index=True)
        
    c='C'
    
    for index,row in temp.iterrows():
        if(c=='C'):
            c='A'
        elif(c=='A'):
            c='B'
        elif(c=='B'):
            c='C'
        if(row['klass']==c):
            temp.at[index,'klass']=1
        else:
            temp.at[index,'klass']=0
            
    return temp

In [ ]:
train = func(train)
dev = func(dev)
trial = func(trial)

In [20]:
test = pd.read_csv('Test Data/subtaskB_test_data.csv')


col_names=['text']
temp=pd.DataFrame(columns=col_names)

for index,row in test.iterrows():
    arow={'text':str(row['FalseSent'])+'<eol>.'+str(row['OptionA'])}
    temp = temp.append(arow, ignore_index=True)
    brow={'text':str(row['FalseSent'])+'<eol>.'+str(row['OptionB'])}
    temp = temp.append(brow, ignore_index=True)
    crow={'text':str(row['FalseSent'])+'<eol>.'+str(row['OptionC'])}
    temp = temp.append(crow, ignore_index=True)
    
test = temp
test.to_json('test.json', orient = 'records')

In [24]:
temp = pd.concat([train, dev, trial])

temp.to_json('train.json', orient = 'records')

In [29]:
import json

def confunc(l):
    for i in l:    
        with open(i, "r") as read_file:
            data = json.load(read_file)
        result = [json.dumps(record) for record in data]
        with open(i, 'w') as obj:
            for i in result:
                obj.write(i+'\n')

In [31]:
temp = pd.concat([train, dev])
temp.to_json('train.json', orient = 'records')
del trial['klass']
trial.to_json('test.json', orient = 'records')
confunc(['train.json', 'test.json'])